# Treinamento da IA
### Arthur Fenili RM 552752
### Enzo Antunes Oliveira RM 553185
### Vinicío Raphael RM 553813

In [21]:
# Importando as bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from random import choice, randint, uniform
from datetime import timedelta

In [22]:
# Carregando os dados
file_path = '/content/Tabela_Treino.csv'
data = pd.read_csv(file_path)

In [23]:
# Garantindo que 'CUSTO_CONSULTA' esteja no formato numérico
data['CUSTO_CONSULTA'] = pd.to_numeric(data['CUSTO_CONSULTA'], errors='coerce')
data['CUSTO_CONSULTA'].fillna(data['CUSTO_CONSULTA'].mean(), inplace=True)

<ipython-input-23-42be65ac4f3a>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['CUSTO_CONSULTA'].fillna(data['CUSTO_CONSULTA'].mean(), inplace=True)


In [24]:
# Verificar conversão do 'CUSTO_CONSULTA'
print("Tipo de CUSTO_CONSULTA:", data['CUSTO_CONSULTA'].dtype)

Tipo de CUSTO_CONSULTA: int64


In [25]:
# Converter 'DATA_CONSULTA' para formato de data
data['DATA_CONSULTA'] = pd.to_datetime(data['DATA_CONSULTA'], format='%d/%m/%Y', errors='coerce')

In [26]:
# Definir os limites de preço para os tratamentos
pricing_limits = {
    "Restauração Dentária": (150, 1800),
    "Tratamento de Canal": (250, 1000),
    "Tratamento Periodontal": (350, 1500),
    "Implantodontia": (1500, 3000),
    "Clareamento Dental": (400, 700)
}

In [27]:
# Função para verificar se o custo está fora da faixa
def cost_out_of_range(row):
    treatment_type = row['DESCRICAO_TRATAMENTO']
    cost = row['CUSTO_CONSULTA']
    if treatment_type in pricing_limits:
        min_price, max_price = pricing_limits[treatment_type]
        if cost < min_price or cost > max_price:
            return 1
    return 0

In [28]:
# Função
data['OUT_OF_RANGE'] = data.apply(cost_out_of_range, axis=1)

In [29]:
# Calcular dias desde a última consulta para o mesmo paciente
data = data.sort_values(by=['CPF', 'DATA_CONSULTA'])
data['DAYS_SINCE_LAST'] = data.groupby('CPF')['DATA_CONSULTA'].diff().dt.days.fillna(9999)
data['RECENT_CONSULTATION'] = np.where(data['DAYS_SINCE_LAST'] <= 2, 1, 0)

In [30]:
# Codificar tratamentos e dentistas
le_treatment = LabelEncoder()
le_dentist = LabelEncoder()

data['TREATMENT_ENCODED'] = le_treatment.fit_transform(data['DESCRICAO_TRATAMENTO'])
data['DENTIST_ENCODED'] = le_dentist.fit_transform(data['NOME_DENTISTA'])

In [31]:
  # Selecionar as variáveis e o alvo
  X = data[['CUSTO_CONSULTA', 'OUT_OF_RANGE', 'RECENT_CONSULTATION', 'TREATMENT_ENCODED', 'DENTIST_ENCODED']]
  y = data['SINISTRO']

  # Dividir o conjunto em treino e teste
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# Inicializando o modelo Random Forest
rf_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)

# Treinando modelo
rf_model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf_model.predict(X_test)

In [33]:
# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [34]:
# Exibir resultados
print("Acurácia do modelo:", accuracy)
print("\nRelatório de Classificação:\n", report)
print("\nMatriz de Confusão:\n", conf_matrix)

Acurácia do modelo: 0.52

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.65      0.65      0.65        69
           1       0.23      0.23      0.23        31

    accuracy                           0.52       100
   macro avg       0.44      0.44      0.44       100
weighted avg       0.52      0.52      0.52       100


Matriz de Confusão:
 [[45 24]
 [24  7]]


In [35]:
# Contar o número de sinistros (True) na coluna SINISTRO
sinistro_count = data['SINISTRO'].sum()
total_entries = data.shape[0]

In [36]:
# Exibição de sinistros
print(f"Total de sinistros indicados: {sinistro_count}")
print(f"Proporção de sinistros: {sinistro_count / total_entries:.2%} do total")

Total de sinistros indicados: 165
Proporção de sinistros: 33.00% do total


# Inserir nova tabelas para saber os sinistros


In [37]:
# Carregando a nova tabela
new_data_path = '/content/Tabela_Sem_Sinistro.csv'
new_data = pd.read_csv(new_data_path)

# Garantir a conversão das colunas
new_data['CUSTO_CONSULTA'] = pd.to_numeric(new_data['CUSTO_CONSULTA'], errors='coerce')
new_data['CUSTO_CONSULTA'] = new_data['CUSTO_CONSULTA'].fillna(new_data['CUSTO_CONSULTA'].mean())
new_data['DATA_CONSULTA'] = pd.to_datetime(new_data['DATA_CONSULTA'], format='%d/%m/%Y', errors='coerce')

# Aplicação das regras
new_data['OUT_OF_RANGE'] = new_data.apply(cost_out_of_range, axis=1)
new_data = new_data.sort_values(by=['CPF', 'DATA_CONSULTA'])
new_data['DAYS_SINCE_LAST'] = new_data.groupby('CPF')['DATA_CONSULTA'].diff().dt.days.fillna(9999)
new_data['RECENT_CONSULTATION'] = np.where(new_data['DAYS_SINCE_LAST'] <= 2, 1, 0)

# Transformação de variáveis categóricas
new_data['TREATMENT_ENCODED'] = le_treatment.transform(new_data['DESCRICAO_TRATAMENTO'])
new_data['DENTIST_ENCODED'] = le_dentist.transform(new_data['NOME_DENTISTA'])

# Selecionar features para previsão e preencher a coluna SINISTRO
X_new = new_data[['CUSTO_CONSULTA', 'OUT_OF_RANGE', 'RECENT_CONSULTATION', 'TREATMENT_ENCODED', 'DENTIST_ENCODED']]
new_data['SINISTRO'] = rf_model.predict(X_new)

# Contagem e proporção de sinistros identificados
sinistro_count_new = new_data['SINISTRO'].sum()
total_entries_new = new_data.shape[0]
proportion_sinistros = sinistro_count_new / total_entries_new

# Exibir resultados
print(f"Total de sinistros indicados: {sinistro_count_new}")
print(f"Proporção de sinistros: {proportion_sinistros:.2%} do total")

# Salvar a nova tabela com a coluna SINISTRO preenchida para análise posterior
output_path = 'Nova_Tabela_Com_Sinistros_Preenchida.csv'
new_data.to_csv(output_path, index=False)

Total de sinistros indicados: 25
Proporção de sinistros: 7.46% do total
